<h1>Bert Implimentation for Que and Ans</h1>

<img src="huggingface.png" alt="huggingface" style="width:1000px;height:400px;">

## Contents
* Overview
* BERT Summary
* BERT Tokenization
* BERT Embaddings
* Understand BERT Outputs
* Embedding layer in BERT
* BERT for QnA
* Testing model performance


## Overview

BERT (Bidirectional Encoder Representations from Transformers) is the state of the art NLP technique released by the google AI research in october 2018. The best thing about BERT is it is open source transfer learning plateform for almost all variety of NLP problem. One of the most common usecase of BERT is question and answer problem where the model requires a passage and question and it generate answer for that question from the passage. In this blog post I will impliment BERT model for Que and Ans task.

Unlike Other NLP task such as Sentiment Analysis, Part of speech tagging or Name entity recognition, BERT model does not require fine tuning for que and ans application as it gives significantly good results on pretrained model.

This blog is continuation of Jay Allamar's [blog](http://jalammar.github.io/illustrated-bert/) where he explained the architecture and woking of BERT model amazingly. I will try to fill the missing dots which is not covered in that blog and I will also share some problems that I faced while implimenting this model for Que and Ans task.

### BERT Summary
If you have gone through this [blog](http://jalammar.github.io/illustrated-bert/) then you must have some idea about query , key , value, self attention , attention heads and positional encoding. Now let's directly go to the problems that you might face while implimenting BERT.

In this blog we will impliment BERT for QnA task. We know that mathematical model do not accept words or sentenses so first we have to convert our sentenses into some numbers so that the model can process it. The sequence of steps is Tokenization , Token ids and Embedding vectors. 

### BERT Tokenization
Tokenization is the first process where we convert a sentenses into list of words such that each word in the list can be mapped to Embadding matrix which will return 768 dimentional or 1024 dimentional vector for bert base and bert large respectively. Now what is Embadding matrix? I will show you this matrix when we load our BERT large model for QnA for now lets first understand how bert tokenizer works and how its different from other embadding techniques.

The vocabulary size is 30,522 for english language and ofcourse this can not cover all english words. Then how BERT tokenizer deal with the words which is not in the vocabulary. Let's see this


In [5]:
from transformers import BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [16]:
tokenizer.vocab_size

30522

You can simply print tokenizer.vocab to see all 30,522 tokens and its corresponding token ids. There are some special tokens which BERT tokenizer uses for special purpose.

In [ ]:
'''
print(tokenizer.vocab)
'''

In [35]:
special_tokens = tokenizer.special_tokens_map
print(special_tokens)
special_token_ids = tokenizer.convert_tokens_to_ids(special_tokens.values())
print(special_tokens.values())
print(special_token_ids)

{'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}
dict_values(['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'])
[100, 102, 0, 101, 103]


* [UNK] : token_id = 100 and used for unknown or misspelled words.
* [CLS] : token_id = 101 By defalt starting token for each input. Last hidden vector corresponding to [CLS] used for classification. We will talk about it later in this blog.
* [SEP] : token_id = 102 used as separation taken between two sentenses. 
* [MASK]:  token_id = 103 used for masking. Masking is a technique that BERT uses while generation sequence of words, at a time it generate one word and mask all other tokens.
* [PAD]:  token_id = 0 used to equal the lenght of sentenses in a batch.

In [36]:
tokens = tokenizer.tokenize('Hey guys! This is Nitesh welcoming you into my blog post.')
print(tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

['hey', 'guys', '!', 'this', 'is', 'ni', '##tes', '##h', 'welcoming', 'you', 'into', 'my', 'blog', 'post', '.']
[4931, 4364, 999, 2023, 2003, 9152, 4570, 2232, 18066, 2017, 2046, 2026, 9927, 2695, 1012]


Ofcourse the word 'Nitesh' which is my name is not there in the vocabulary as I am not a famous personality. One thing to notice that when a word is not there in the vocabulary then the tokenizer breaks down the word into subwords like in this case 'Nitesh' --> 'ni' + '##tes' + '##h' . What are those # tags?

BERT tokenizer uses two # tags to identify that the token containing # tags are not actual word rather it is extention of previous non # tag token. let's see one other eg.


In [37]:
tokens = tokenizer.tokenize('Dewangan')
print(tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

['dew', '##anga', '##n']
[24903, 18222, 2078]


Here also the word 'Dewangan' which is my surname is break down into three tokens. You must be thinking that how breaking down unknown words into small token could actually works because the model generate separte hidden vector for each token. I will discuss about this topic when we will load the model. Till now we have understan d about BERT token and token ids. Now let's understand BERT Embaddings.



### BERT Embaddings
We have number of word embadding techniques such as bag of word, tfidt embadding and word2vec embadding technique. Out of those word2vec is advance word embadding technique which take care of semantic meaning of words. We will compare word2vec embadding technique with BERT Embadding and understand how geniously BERT generate word embadding by considering context word into consideration.

Though the state of the art word2vec embadding technique holds semantic meaning of words but it does not take care of context word. lets understand it with an eg. The sentence 'After stealing money from the bank vault, the bank robber was seen fishing on the Mississippi river bank.' uses the word 'bank' three times first and second 'bank' is reffering a "finacial establishment that uses money deposited by customers for investment" and third 'bank' is reffering river edge or river site. Word2vec embadding gives same embadding vector for all three 'bank' where BERT uses context word and will generate different embadding vector for same word 'bank'.   

To know more about BERT Embadding you can visit this [blog](https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/)

In [64]:
# Define a new example sentence with multiple meanings of the word "bank"
text = "After stealing money from the bank vault, the bank robber was seen " \
       "fishing on the Mississippi river bank."
# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"
# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)
# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Display the words with their indeces.
print(f'Number of tokens : {len(tokenized_text)}')
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))


Number of tokens : 22
[CLS]           101
after         2,044
stealing     11,065
money         2,769
from          2,013
the           1,996
bank          2,924
vault        11,632
,             1,010
the           1,996
bank          2,924
robber       27,307
was           2,001
seen          2,464
fishing       5,645
on            2,006
the           1,996
mississippi   5,900
river         2,314
bank          2,924
.             1,012
[SEP]           102


Our actual task is QnA but I want to show that embadding for all three 'bank' will be different. So for that I will use bert-base-uncased just for this purpose you don't have to do this.

In [41]:
import torch
from transformers import BertTokenizer, BertModel

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Mark each of the 22 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

print (segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [42]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [43]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [44]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

In [57]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.4964, -0.1831, -0.5231,  ..., -0.1902,  0.3738,  0.3964],
         [-0.1323, -0.2762, -0.3495,  ..., -0.4567,  0.3786, -0.1096],
         [-0.3626, -0.4002,  0.0676,  ..., -0.3207, -0.2709, -0.3004],
         ...,
         [ 0.2961, -0.2856, -0.0382,  ..., -0.6056, -0.5163,  0.2005],
         [ 0.4878, -0.0909, -0.2358,  ..., -0.0017, -0.5945, -0.2431],
         [-0.2517, -0.3519, -0.4688,  ...,  0.2500,  0.0336, -0.2627]]]), pooler_output=tensor([[-0.6031, -0.3342, -0.7174,  0.3347,  0.5145, -0.1722,  0.4502,  0.2768,
         -0.3769, -0.9998, -0.3657,  0.7535,  0.9817, -0.0192,  0.7959, -0.3459,
         -0.1338, -0.3026,  0.1097,  0.5836,  0.5736,  0.9999,  0.1798,  0.1845,
          0.2250,  0.9109, -0.5653,  0.8616,  0.8994,  0.7423, -0.2525,  0.0394,
         -0.9894, -0.1331, -0.7763, -0.9826,  0.2223, -0.6115,  0.1941,  0.0177,
         -0.7634,  0.2312,  0.9999, -0.7000,  0.4623, -0.2202, -1.0000,  0.

Since we enabled output_hidden_states = True that's why we will get three outputs. 

### Understand BERT Outputs

In [66]:
print(f'first output = last_hidden_state with shape {outputs[0].shape}')
print(f'second output = pooler_output with shape {outputs[1].shape}')
print('='*50)
print('third output')
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

first output = last_hidden_state with shape torch.Size([1, 22, 768])
second output = pooler_output with shape torch.Size([1, 768])
third output
Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 22
Number of hidden units: 768


Bert base has 12 bert layers and for each bert layer it gives embaddings for tokens. we are getting number of layers = 13 because the model adds one additional embadding layer at very beggining.
Now lets understand the outputs

<img src="BertOutputs.png" alt="huggingface" style="width:800px;height:400px;">

* Pooler Output : Creator of BERT model said that pooler_output which corresponds to last hidden state of [CLS] token does not have any interpretability but it is best choice as an input for a classifer that can be fine tuned on separate dataset. The main reason behind this is the way it pretrained. We must not forget that we are using BERT model as transfer learning which is already pretrained on large amount of data. [CLS] token is always used at starting token while performing pretraing using masked language model and next sentense prediction that's why pooler output which itself don't have any interpretability but it captures all the information for a perticular input.
* Hidden_state : The second reason to love this model is its hidden state. BERT application is not limited to use pooler output to fine tune the classifier but one can also explore the advantages by using its hidden states. Though there is not hard and fast rule to which hidden state to use for embadding but depending on dataset and tasks one can try this followings combination

<img src="hiddenembadding.png" alt="huggingface" style="width:800px;height:400px;">

Just for cross check first output which is last hidden state and last index of third output which is all hidden_state should be same

In [60]:
outputs[0]

tensor([[[-0.4964, -0.1831, -0.5231,  ..., -0.1902,  0.3738,  0.3964],
         [-0.1323, -0.2762, -0.3495,  ..., -0.4567,  0.3786, -0.1096],
         [-0.3626, -0.4002,  0.0676,  ..., -0.3207, -0.2709, -0.3004],
         ...,
         [ 0.2961, -0.2856, -0.0382,  ..., -0.6056, -0.5163,  0.2005],
         [ 0.4878, -0.0909, -0.2358,  ..., -0.0017, -0.5945, -0.2431],
         [-0.2517, -0.3519, -0.4688,  ...,  0.2500,  0.0336, -0.2627]]])

In [62]:
outputs[2][-1]

tensor([[[-0.4964, -0.1831, -0.5231,  ..., -0.1902,  0.3738,  0.3964],
         [-0.1323, -0.2762, -0.3495,  ..., -0.4567,  0.3786, -0.1096],
         [-0.3626, -0.4002,  0.0676,  ..., -0.3207, -0.2709, -0.3004],
         ...,
         [ 0.2961, -0.2856, -0.0382,  ..., -0.6056, -0.5163,  0.2005],
         [ 0.4878, -0.0909, -0.2358,  ..., -0.0017, -0.5945, -0.2431],
         [-0.2517, -0.3519, -0.4688,  ...,  0.2500,  0.0336, -0.2627]]])

The word bank occures at index 6th , 10th and 19th. first we will see that embaddings are different or not and then we will examine whether it hold the contextual information or not by calculation cosin distance. Though the embadding vector is 768 dimentional but I will print first five just for comparision.

In [82]:
for i, token in enumerate(tokenized_text):
    if token == 'bank':
        print(f'token : {token} , Embaddings : {outputs[0][0][i][0:5]}')

token : bank , Embaddings : tensor([ 0.9001, -0.5380, -0.1669,  0.2242,  0.6897])
token : bank , Embaddings : tensor([ 0.7977, -0.5217, -0.1984,  0.1890,  0.5941])
token : bank , Embaddings : tensor([ 0.2961, -0.2856, -0.0382,  0.1674,  0.7713])


In [85]:
from scipy.spatial.distance import cosine

similarity12 = 1 - cosine(outputs[0][0][6],outputs[0][0][10])
print(f'cosine similarity between 1st and 2nd bank {similarity12:.4f}')

similarity13 = 1 - cosine(outputs[0][0][6],outputs[0][0][19])
print(f'cosine similarity between 1st and 3rd bank {similarity13:.4f}')

cosine similarity between 1st and 2nd bank 0.9527
cosine similarity between 1st and 3rd bank 0.6989


We can see that BERT generates different embaddings for same word 'bank' and also holds the contextual information as cosine similarity between first and second bank is high where it is low for first and third bank.

Till now we have understand BERT tokens , token ids , BERT Embaddings and model outputs including last hidden output , pooled output and hidden states and we also seen that BERT Embadding takes care of contextual information.

Now let's start our main task which is QnA. First we will load the model

### Embedding layer in BERT

In [4]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [100]:
model

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12,

In this blog I will talk about sections in Embadding layer and not the bert layer as I am assuming that reader already know about self attention , attention heads , query , key , value , layer_norm and finaly feed forward output layer.
word_embaddings : At the beggining there is a section in embadding layer called word_embedding which has a matrix of size (30522,1024). This matrix can be thought of as a dictionary where keys are token ids and values are 1024(bert-large) dimentional vector. Note that this matrix is trainable and model will update vectors (by back propagation ) corresponding to only those tokens which are present in the traing data while fine tuning the model. 

Remember that this embadding matrix is just a dictionary which gives same vectors for same token, it is last hidden layer output where we can expect different embadding for same word that captures context information.

position_embeddings : BERT-base and BERT-large both can accept sentence whose token lengths is less than 512. 
so before feeding inputs to the model ensure that the length of tokens should be less than or equal to 512. Unlike LSTM or RNN we feed entire sequence to the model in one go so the model has to do something to understand the sequence information(which word come before and which come after). Model adds possitional embedding to each tokens to do so. I think that positional matrix is not trainable because it is used just to tell model that which word comes before and which comes later and they did lots of research to come up with this matrix.


token_type_embeddings: Remember that bert model is pretrained on large datset using masked language model and next sentence prediction. So by default model expect two sentences as input which are separated by [SEP] token though task like classification and sentiment analysis do not feed two sentences as input. In task like classification we feed additional attention mask that tells model that which token is coming from sentences and which are just padding tokens to match the length. In task like QnA where we have to feed two sentences first as passage and second for que so we must provide additional segment ids (0's for first segment and 1's for second segmnet). we will see this in detail when we pass a input to our QnA model.

So what is this token_type_embeddings (2,1024)? 
we feed additional segment ids but the token embaddings are 1024 dimentional vector. I think that token_type_embeddings provides 1024 dimentional zero vector for segment id ==0 and 1024 dimentional vector with ones for segment id = 1. 

Before talking about last linear layer let's take our first eg. to test the model BERT-large-for-Que-Ans.
BertTokenizer provides inbuilt module called 'encode' that takes pair of question and passage sentense and perform three task.
first it performs tokenization to get the tokens from vocabulary for entire input.
then it places [CLS] token at the start and [SEP] token at the end of first sequence and after the end of last sequence.
then it gives token ids for all the tokens.

### BERT for QnA

In [3]:
from transformers import BertTokenizer

question = "Which mechnism that BERT use?"

answer_text = ''' BERT makes use of Transformer, an attention mechanism that learns contextual relations between 
        words (or sub-words) in a text. In its vanilla form, Transformer includes two separate 
        mechanisms — an encoder  that reads the text input and a decoder that produces a prediction 
        for the task. Since BERT’s goal is to generate a language model, only the encoder mechanism 
        is necessary. The detailed workings of Transformer are described in a paper by Google.'''

# Apply the tokenizer to the input text, treating them as a text-pair.

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
input_ids = tokenizer.encode(question, answer_text)
print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 106 tokens.


This section of code can be used to visualize token and its corresponding ids

In [118]:
# tokenizer's behavior, let's also get the token strings and display them.
tokens = tokenizer.convert_ids_to_tokens(input_ids)

# For each token and its id...
for token, id in zip(tokens, input_ids):
    
    # If this is the [SEP] token, add some space around it to make it stand out.
    if id == tokenizer.sep_token_id:
        print('')
    
    # Print the token string and its ID in two columns.
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')

[CLS]           101
which         2,029
me            2,033
##ch          2,818
##nism       28,113
that          2,008
bert         14,324
use           2,224
?             1,029

[SEP]           102

bert         14,324
makes         3,084
use           2,224
of            1,997
transform    10,938
##er          2,121
,             1,010
an            2,019
attention     3,086
mechanism     7,337
that          2,008
learns       10,229
context       6,123
##ual         8,787
relations     4,262
between       2,090
words         2,616
(             1,006
or            2,030
sub           4,942
-             1,011
words         2,616
)             1,007
in            1,999
a             1,037
text          3,793
.             1,012
in            1,999
its           2,049
vanilla      21,161
form          2,433
,             1,010
transform    10,938
##er          2,121
includes      2,950
two           2,048
separate      3,584
mechanisms   10,595
—             1,517
an            2,01

Now we have to generate segment ids to tell the model that which token id belongs to first sequence and which token id belongs to second sequence.

In [119]:
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)
print(segment_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [120]:
len(segment_ids) == len(input_ids)

True

Now we have processed our input (question and passage) into the form that our BERT model expects the input to be. Let's convert processed input into tensor in feed into the model.

In [123]:
# Run our example through the model.
import torch
outputs = model(
            torch.tensor([input_ids]), # The tokens representing our input text.
            token_type_ids=torch.tensor([segment_ids]),# The segment IDs to differentiate question from answer_text
            return_dict=True
                ) 

outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-5.9297, -8.2512, -8.2334, -9.1164, -7.8028, -7.8494, -8.5925, -8.2909,
         -9.3075, -5.9297,  0.9577, -3.3898, -4.1839, -4.8172,  6.8211, -1.1268,
         -4.5010,  1.2108,  0.5921, -2.3455, -5.4775, -3.0783, -4.2873, -7.1977,
         -5.5380, -6.3641, -5.0001, -7.8545, -7.7550, -5.7282, -8.6403, -6.3636,
         -7.2707, -7.0612, -6.2764, -3.3167, -5.9297, -4.0268, -4.3900, -1.6861,
         -5.1395, -6.9378,  3.0385, -4.0382, -5.7145, -3.7573, -6.3599, -5.2701,
         -7.0734, -2.5215, -1.6866, -5.7555, -5.9852, -7.0097, -5.1551, -7.8404,
         -5.6063, -6.1879, -7.9365, -4.8895, -2.9879, -6.2326, -7.8125, -6.4680,
         -7.9348, -5.5899, -8.5970, -8.7157, -6.4139, -6.9194, -5.1876, -4.0657,
         -8.0904, -8.7001, -5.9679, -8.7346, -6.2633, -4.2040, -5.9619, -3.7574,
         -5.8462, -8.6115, -4.1992, -4.1589, -2.2915, -6.3091, -6.5090, -6.5777,
         -8.5397, -6.8958, -8.5823, -7.2457, -6.5215, -6

In [124]:
print(outputs[0].shape , outputs[1].shape)

torch.Size([1, 106]) torch.Size([1, 106])


Our output is list of two tensor each of those is 106 dimentional vector. To understand this output now let's talk about last linear layer of our model.

(take screenshot of last linear layer)(img)

The last linear layer takes input as 1024 dimentional vector and generate two outputs. After carefully reading this [code](https://huggingface.co/transformers/_modules/transformers/models/bert/modeling_bert.html#BertForQuestionAnswering)  provided by huggingface library I got to know that this last linear layer takes input from last hidden state of every input tokens and for each input token this last linear layer generates two outputs. The value of 
first output indicate the chance of corresponding token to be the start index of the final answer. Large positive value means high chance that corresponding token is the starting index and large negative means low chance to be the starting index of the final answer. Same idea applicable for second output except it is related to end index of the final answer.

After this everything is very simple, you just have to find the index of largest value in the first output and assign its corresponding token as starting index  and same for second output to get the last index of the final answer.

In [122]:
start_scores = outputs.start_logits
end_scores = outputs.end_logits

answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

# Combine the tokens in the answer and print it out.
answer = ' '.join(tokens[answer_start:answer_end+1])

print('Answer: "' + answer + '"')

Answer: "transform ##er"


Instead of indexing of outputs we can also use start_logits (index 0) and end_logits (index 1). We can remove this # tags and combine the words.

In [137]:
answer = answer.replace(' ##','')
print('Question : Which mechnism that BERT use?')
print('Expected Answer : transformer as attention')
print(f'Predicted Answer : {answer}')

Question : Which mechnism that BERT use?
Expected Answer : transformer as attention
Predicted Answer : transformer


Now you must be appreciating the brilliant idea of creator of this BERT model that how geniously they uses the idea of attention mechnism because by adding one more linear layer only the model is able to predict starting token and ending token of the final answer based on perticular question that means the model very well understands the meaning of a question and able to find the answer in given passage.

Example provided by huggingface for que and ans didn't explian about the situation where start index > end index. In my case I didn't get those situation but if it occures then I think that instead of largest value index we can use 2nd largest value index depending on situation.

Now lets make helper function that takes question ans passage as input and will return final answer and test the model for other questions.

### Testing model performance

In [7]:
import torch
def get_ans(que,passage):
    
    input_ids = tokenizer.encode(que, passage)
    
    # tokenizer's behavior, let's also get the token strings and display them.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    
    outputs = model(
            torch.tensor([input_ids]), # The tokens representing our input text.
            token_type_ids=torch.tensor([segment_ids]),# The segment IDs to differentiate question from answer_text
            return_dict=True
                )
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
   
    # Combine the tokens in the answer and print it out.
    answer = ' '.join(tokens[answer_start:answer_end+1])
    answer = answer.replace(' ##','')
    return answer

    
    

In [8]:
passage = 'Computer vision is evolving rapidly day-by-day. Its one of the reason is deep learning.\
           When we talk about computer vision, a term convolutional neural network( abbreviated as CNN)\
           comes in our mind because CNN is heavily used here. Examples of CNN in computer vision are \
           face recognition,image classification etc. It is similar to the basic neural network. CNN \
           also have learnable parameter like neural network i.e, weights, biases etc.'

question = 'what is the reason behind evolution of Computer vision'

answer = get_ans(question,passage)
print('Expected Answer : deep learning')
print(f'Predicted Answer : {answer}')

Expected Answer : deep learning
Predicted Answer : deep learning


In [148]:
question = 'what is abbriviated as CNN'
answer = get_ans(question,passage)
print('Expected Answer : convolutional neural network')
print(f'Predicted Answer : {answer}')

Expected Answer : convolutional neural network
Predicted Answer : convolutional neural network


In [149]:
question = 'what are the application of CNN'
answer = get_ans(question,passage)
print(f'Predicted Answer : {answer}')

Predicted Answer : face recognition , image classification


In [140]:
print(passage)

Computer vision is evolving rapidly day-by-day. Its one of the reason is deep learning. When we talk about computer vision, a term convolutional neural network( abbreviated as CNN) comes in our mind because CNN is heavily used here. Examples of CNN in computer vision are face recognition, image classification etc. It is similar to the basic neural network. CNN also have learnable parameter like neural network i.e, weights, biases etc.
